In [2]:
import torch

In [7]:
BATCH_SIZE = 5
T=10

In [8]:
 t = torch.randint(0, T, (BATCH_SIZE,)).long()

In [2]:
import librosa
import numpy as np
from utills import *

Total files found: 44257
train dataset length: 354
Total files found: 44257
test dataset length: 88
Completed


In [17]:
def invert_magnitude_to_audio(magnitude, hop_length):
    # Initialize a random phase
    phase = np.random.randn(*magnitude.shape)
    
    # Reconstruct the waveform using Griffin-Lim algorithm
    for i in range(100):
        # Synthesize complex spectrogram
        complex_spectrogram = magnitude * np.exp(1j * phase)
        
        # Inverse STFT to reconstruct waveform
        waveform = librosa.istft(complex_spectrogram, hop_length=hop_length)
        
        # Compute the new phase
        _, phase = librosa.magphase(librosa.stft(waveform, hop_length=hop_length))
    
    return waveform

In [21]:
waveform, sample_rate = librosa.load("data/VCTK-Corpus/wav48/p226/p226_002.wav", sr=16000)

In [5]:
stft = get_stft(waveform)

In [7]:
waveform2, sample_rate = librosa.load("data/VCTK-Corpus/wav48/p310/p310_002.wav", sr=16000)

In [8]:
stft2 = get_stft(waveform2)

In [30]:
hop_length = 512
n_fft = 2048
stft = librosa.stft(waveform, n_fft=n_fft, hop_length=hop_length)
magnitude, phase = librosa.magphase(stft)



In [31]:
# Reconstruct the waveform using the inverse STFT
waveform_reg = librosa.istft(magnitude * phase, hop_length=hop_length)


In [25]:
print(stft.shape)
print(stft2.shape)

(1025, 149)
(1025, 185)


In [34]:
def preprocess_audio(audio_path, noise_path_list, sampling_rate):
    # Load the audio file
    waveform, _ = librosa.load(audio_path, sr=sampling_rate)
    
    # Load a random noise file
    random_noise = noise_path_list #np.random.choice(noise_path_list)
    noise, _ = librosa.load(random_noise, sr=sampling_rate)
    
    # Define desired length (2 seconds)
    desired_length = sampling_rate * 2
    
    # If the waveform is shorter than the desired length, pad it with zeros
    if len(waveform) < desired_length:
        padding = desired_length - len(waveform)
        waveform = np.pad(waveform, (0, padding), 'constant')
    
    # If the noise is shorter than the desired length, pad it with zeros
    if len(noise) < desired_length:
        padding = desired_length - len(noise)
        noise = np.pad(noise, (0, padding), 'constant')
    
    # If the waveform is longer than 2 seconds, take the middle 2 seconds
    if len(waveform) > desired_length:
        start = (len(waveform) - desired_length) // 2
        waveform = waveform[start:start + desired_length]
    
    # If the noise is longer than 2 seconds, take the first 2 seconds
    if len(noise) > desired_length:
        noise = noise[:desired_length]
    
    return waveform, noise

In [37]:
waveform,noise = preprocess_audio("data/VCTK-Corpus/wav48/p310/p310_002.wav","data/noise/sample-2.wav",16000)

In [40]:
stft = get_stft(waveform)
stft.shape

(1025, 63)

In [39]:
type(waveform)

numpy.ndarray

In [44]:
hop_length = 512

# Reconstruct the waveform using the inverse STFT
waveform_regenerate = librosa.istft(magnitude * phase, hop_length=hop_length)

In [45]:
waveform_regenerate.shape

(75776,)

In [32]:
import soundfile as sf
sf.write("test_regerate.wav", waveform_reg, 16000)

In [36]:
import soundfile as sf
sf.write("test.wav", waveform, 16000)